In [138]:
import os
import subprocess
import pandas as pd
import re

import yt_dlp

import whisper
from deep_translator import GoogleTranslator

In [141]:
result = subprocess.run(["ffmpeg", "-version"], capture_output=True, text=True)
print(" ".join(result.stdout.split()[:3]))

ffmpeg version 7.1


In [146]:
def download_audio(url, filename_base, verbose=False, noprogress=False):
    # yt-dlp options to download best audio, then convert to MP3 with a defined bitrate
    ydl_options = {
        'format': 'bestaudio/best',  # best available audio
        'outtmpl': filename_base,
        'postprocessors': [
            {
                'key': 'FFmpegExtractAudio',
                'preferredcodec': 'mp3',
                'preferredquality': '128',  # 128 kbps
            },
        ],
    }
    # Suppress output if needed
    if not verbose:
        ydl_options['quiet'] = True
        ydl_options['no_warnings'] = True  # Optional: Hide warnings
    if noprogress:
        # ydl_options['noprogress'] = T
        ydl_options['progress_hooks'] = []  # Disable progress bar updates

    with yt_dlp.YoutubeDL(ydl_options) as ydl:
        print(f"Downloading: {url}")
        ydl.download([url])

    print("Download complete!")

filename_base = "my_dutch_audio"
url = "https://www.youtube.com/watch?v=NgdXuqrRaDU"
download_audio(url=url, filename_base=filename_base, verbose=False, noprogress=True)

Downloading: https://www.youtube.com/watch?v=NgdXuqrRaDU
Download complete!                                       


In [143]:
def get_file_size(filename_base):
    audio_filename = filename_base + '.mp3'
    return  os.path.getsize(audio_filename) / (1024 * 1024)
print(f"Audio file size: {get_file_size(filename_base):.2f} MB")

Audio file size: 4.65 MB


In [145]:
# Load a model. 
# "small", "medium", or "large" are typical. "small" is faster but less accurate; "large" is more accurate but slower.
model_size = "small" 
model = whisper.load_model(model_size)

print(f"Loaded Whisper model: {model_size}")


Loaded Whisper model: small


In [151]:
def transcribe_audio(filename_base):
    audio_filename = filename_base + '.mpg'
    result = model.transcribe(audio_filename, language="nl")  # 'nl' is for Dutch
    dutch_text = result["text"]
    return dutch_text

# dutch_text = transcribe_audio(filename_base)
print(
    "Dutch transcription (first 100 words):\n",
     " ".join(dutch_text.split()[:100])+"..."
)

Dutch transcription (first 100 words):
 *** Juf Roos en haar kinderen zijn klaar om te beginnen aan een nieuwe schooldag. Waar zullen ze het vandaag over gaan hebben? Ben jij ook zo nieuwsgierig? Ga dan maar lekker zitten voor een mooi verhaal van Juf Roos. *** Hoi, gijs. Hallo, je vrouw. Het is weer een mooie, nieuwe dag. Het is een hele mooie, speciale dag. Inderdaad. Het is mooi weer, de zon schijnt, het is lekker warm buiten. Het is niet. Wat bedoel je dan? Dat het vandaag een hele mooie dag is om iets leuk te doen. Maar we doen altijd leuke dingen in...


In [152]:
def translate(dutch_text):
    translator = GoogleTranslator(source="nl", target="en")
    return translator.translate(dutch_text)

english_text = translate(dutch_text)
print(
    "English translation (first 100 words):\n",
     " ".join(english_text.split()[:100])+"..."
)

English translation (first 100 words):
 *** Miss Roos and her children are ready to start a new school day. What will they be talking about today? Are you so curious? Then sit comfortably for a nice story by Miss Roos. *** Hi, Gijs. Hello, your wife. It is a nice new day. It is a very nice, special day. Indeed. The weather is nice, the sun is shining, it is nice and warm outside. It's not. What do you mean then? That today is a very nice day to do something fun. But we always do fun things in my mill. I mean today it...


In [83]:
def test(dutch_text=None, english_text=None):
    if not dutch_text or not english_text:
        # Example Dutch and English text (replace these with your actual transcriptions)
        dutch_text = """Dit is een voorbeeldzin in het Nederlands. Ik ben bezig met het testen van Whisper! De resultaten lijken behoorlijk accuraat te zijn?"""

        english_text = """This is an example sentence in Dutch. I am testing Whisper! The results seem to be quite accurate?"""

    # Regular expression to match sentence-ending punctuation and keep it
    pattern = r'([.!?]+)'

    def split_and_recombine(text):
        """Splits the text into sentences while keeping punctuation."""
        split_text = re.split(pattern, text)
        sentences = ["".join(split_text[i:i+2]).strip() for i in range(0, len(split_text)-1, 2)]
        return sentences

    # Apply sentence splitting to both Dutch and English text
    dutch_sentences = split_and_recombine(dutch_text)
    english_sentences = split_and_recombine(english_text)

    # Ensure both lists have the same length (fill with empty strings if needed)
    max_len = max(len(dutch_sentences), len(english_sentences))
    dutch_sentences += [""] * (max_len - len(dutch_sentences))
    english_sentences += [""] * (max_len - len(english_sentences))

    # Create DataFrame
    df = pd.DataFrame({"Dutch (Original)": dutch_sentences, "English (Translation)": english_sentences})

    return df

# Run function and display DataFrame
df = test(dutch_text=dutch_text, english_text=english_text)


In [87]:
for i in range(min(len(df), 5)):
    print(f'{df.iloc[i].iloc[0]} | {df.iloc[i].iloc[1]}')

*** Juf Roos en haar kinderen zijn klaar om te beginnen aan een nieuwe schooldag. | *** Miss Roos and her children are ready to start a new school day.
Waar zullen ze het vandaag over gaan hebben? | What will they be talking about today?
Ben jij ook zo nieuwsgierig? | Are you so curious?
Ga dan maar lekker zitten voor een mooi verhaal van Juf Roos. | Then sit comfortably for a nice story by Miss Roos.
*** Hoi, gijs. | *** Hi, Gijs.


In [90]:
df.head(len(df))

,Dutch (Original),English (Translation)
0,*** Juf Roos en haar kinderen zijn klaar om te...,*** Miss Roos and her children are ready to st...
1,Waar zullen ze het vandaag over gaan hebben?,What will they be talking about today?
2,Ben jij ook zo nieuwsgierig?,Are you so curious?
3,Ga dan maar lekker zitten voor een mooi verhaa...,Then sit comfortably for a nice story by Miss ...
4,"*** Hoi, gijs.","*** Hi, Gijs."
...,...,...
78,Dat was weer een mooi verhaal van je vroos.,That was another nice story of your voos.
79,Kijk je jullie de volgende keer weer?,Are you watching you again next time?
80,Dag dag!,Day day!
81,"Allemaal zwaaien, tot de volgende keer.","All waving, see you next time."


In [86]:
from IPython.display import display
display(df)


,Dutch (Original),English (Translation)
0,*** Juf Roos en haar kinderen zijn klaar om te...,*** Miss Roos and her children are ready to st...
1,Waar zullen ze het vandaag over gaan hebben?,What will they be talking about today?
2,Ben jij ook zo nieuwsgierig?,Are you so curious?
3,Ga dan maar lekker zitten voor een mooi verhaa...,Then sit comfortably for a nice story by Miss ...
4,"*** Hoi, gijs.","*** Hi, Gijs."
...,...,...
78,Dat was weer een mooi verhaal van je vroos.,That was another nice story of your voos.
79,Kijk je jullie de volgende keer weer?,Are you watching you again next time?
80,Dag dag!,Day day!
81,"Allemaal zwaaien, tot de volgende keer.","All waving, see you next time."
